In [34]:
import pandas as pd
from sys import argv
from datetime import datetime
import pymysql
import inspect, re
import os
#from tqdm.notebook import tqdm
#from tkinter.filedialog import askopenfilename

def logger(*args):
    """
        Print log message with excecuting file name, timestamp.
        *** All parameters should be on string-type!
    """
    try: __file__
    except NameError:
        message = f"[{str(datetime.now())}] {str(' '.join(args))}"
    else:
        message = f"[{__file__} {str(datetime.now())}] {str(' '.join(args))}"
    print(message)

logger("import & definition ends successfully.")

[2021-05-26 22:43:41.570822] import & definition ends successfully.


In [25]:
originPath = globals()['_dh'][0]
os.chdir(originPath)
#currentPath = os.getcwd()
targetFolderList = ['2020년자료', '2021년자료']
dataset = dict()
#dataset = {fName: {} for fName in targetFolderList}

for folderName in targetFolderList:
    targetDir = f"{originPath}/{folderName}"
    os.chdir(targetDir)
    logger(f'Working directory changed to {targetDir}')
    fileList = os.listdir(targetDir)
    for fileName in fileList:
        if fileName in dataset:
            dataset[fileName] = pd.concat([dataset[fileName], pd.read_csv(f"{targetDir}/{fileName}")], ignore_index=True)
        else:
            if fileName.endswith('.csv') and os.path.isfile(fileName):
                try:
                    df = pd.read_csv(fileName)
                    if len(df) > 0: dataset[fileName] = df
                    else: raise Exception
                except:
                    logger(f'Empty or wrong file [{fileName}] detected. jump to next file')
            else: continue
else:
    logger('Reading data from files complete.')

[2021-05-26 22:40:04.243355] Working directory changed to /Users/youngdae/Documents/미세먼지와 폭염 대응 그린쉘터 최적 이용망 분석 기술 개발3(국립산립과학원)/210526_측정망 DB 구축/측정넷자료/2020년자료
[2021-05-26 22:40:04.570681] Working directory changed to /Users/youngdae/Documents/미세먼지와 폭염 대응 그린쉘터 최적 이용망 분석 기술 개발3(국립산립과학원)/210526_측정망 DB 구축/측정넷자료/2021년자료
[2021-05-26 22:40:04.613958] Empty or wrong file [Nafion-태안_산단.csv] detected. jump to next file
[2021-05-26 22:40:04.933637] Empty or wrong file [BVOC-태안_산단.csv] detected. jump to next file
[2021-05-26 22:40:04.934107] Empty or wrong file [Met_태안_산단.csv] detected. jump to next file
[2021-05-26 22:40:04.954094] Empty or wrong file [AVOC-태안_산단.csv] detected. jump to next file
[2021-05-26 22:40:05.023373] Reading data from files complete.


In [26]:
columnSet = {'pm': ['stationName', 'measuredDatetime', 'PM10', 'PM25', 'PM1'],
             'weather': ['stationName', 'measuredDatetime', 'temperature', 'RelativeHumidity', 'windSpeed', 'windDirection']}

data_AVOC = pd.DataFrame(columns=columnSet['pm'])
data_BVOC = pd.DataFrame(columns=columnSet['pm'])
data_Nafion = pd.DataFrame(columns=columnSet['pm'])
data_weather = pd.DataFrame(columns=columnSet['weather'])

dSet = {'AVOC': {}, 'BVOC': {}, 'weather': {}, 'Nafion': {}}

# stationName 추가 및 컬럼명 변경
for key in dataset:
    if key.startswith('AVOC-') or key.startswith('BVOC-'):
        dataset[key].insert(0, "stationName", key[5:-4], True)
        dataset[key].columns = columnSet['pm']
        if key.startswith('AVOC-'): 
            dSet['AVOC'][key] = dataset[key]
        else:
            dSet['BVOC'][key] = dataset[key]
    elif key.startswith('Met_'):
        dataset[key].insert(0, "stationName", key[4:-4], True)
        dataset[key].columns = columnSet['weather']
        dSet['weather'][key] = dataset[key]
    elif key.startswith('Nafion-'):
        dataset[key].insert(0, "stationName", key[7:-4], True)
        dataset[key].columns = columnSet['pm']
        dSet['Nafion'][key] = dataset[key]
else:
    dataset, dSet = dSet, dataset
    logger('Data classification complete.')

for key in dataset:
    vars()['data_'+key] = pd.concat(list(dataset[key].values()), ignore_index=True)
else:
    logger('Data merged successfully.')

[2021-05-26 22:40:06.796342] Data classification complete.
[2021-05-26 22:40:06.874638] Data merged successfully.


In [35]:
os.chdir(originPath)

if os.path.isdir('nifos_data'):
    pass
else:
    os.makedirs('nifos_data')

for dataFile, fileName in zip((data_AVOC, data_BVOC, data_Nafion, data_weather), ['AVOC', 'BVOC', 'Nafion', 'weather']):
    dataFile.to_csv(f"{originPath}/nifos_data/{fileName}.csv")
else:
    logger('data export complete.')


[2021-05-26 22:44:17.627798] data export complete.
